In [1]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import grammar as lt
from collections import Counter
from tqdm import tqdm
from joblib import Parallel, delayed
from spacy import load
import re
import yaml
from pymongo import MongoClient, UpdateOne
from plotqa import match_and_generate
nlp = load('en_core_web_sm')

2023-03-08 12:12:07.826353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 12:12:08.470934: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-08 12:12:08.470981: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-08 12:12:08.470986: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
client = MongoClient()
db = client.plotqa
G = [d for d in db.captions.find({"matches.0": {"$exists": True}})]
len(G)

19887

In [3]:
ctr = lt.err_counter(G, top=False)
msg = ctr.most_common(1)[0][0]

In [5]:
services = lt.filter_lt_messages(G, message=msg)
services[0]

{'_id': 370713,
 'question_string': 'What is the average amount collected by government from taxes on goods and services in Azerbaijan per year?',
 'template': 'arithmetic',
 'answer': 2597133333.3333335,
 'type': 'dot',
 'template_id': 35,
 'caption': 'The mean per year amount collected by government from taxes on goods and services in Azerbaijan is 2597133333.3333335.',
 'matches': [{'message': 'With the plural noun ‘services’, the verb inflection ‘is’ is not correct.',
   'shortMessage': 'Agreement error',
   'replacements': [{'value': 'are'},
    {'value': 'was'},
    {'value': 'were',
     'shortDescription': "second-person singular simple past of 'be', e.g. 'you were'"}],
   'offset': 96,
   'length': 2,
   'context': {'text': '...xes on goods and services in Azerbaijan is 2597133333.3333335.',
    'offset': 43,
    'length': 2},
   'sentence': 'The mean per year amount collected by government from taxes on goods and services in Azerbaijan is 2597133333.3333335.',
   'type': {'ty

In [7]:
qids = [s['_id'] for s in services]
sents = [s['caption'] for s in services]
roots = []
for qid, doc in zip(qids, nlp.pipe(sents)):
    roots.append((qid, *lt.find_root_nsubj(doc)))

IndexError: tuple index out of range

In [8]:
doc

The mean per year amount collected by government from taxes on goods and services in Azerbaijan is 2597133333.3333335.

In [10]:
root = [c for c in doc if c.dep_ == 'ROOT'][0]
root

is

In [15]:
[c.pos_ for c in list(root.children)]

['ADJ', 'NUM', 'PUNCT']

In [4]:
schools = lt.filter_lt_messages(G, message='schools')
len(schools)

1788

In [8]:
qids = [r['_id'] for r in schools]
sents = [r['caption'] for r in schools]

def fix(x):
    try:
        y = lt.fix_verb_subject_agreement(x)
    except (ValueError, IndexError):
        print(x)
        y = x.text
    return y


fixed = [{'_id': qid, 'caption': fix(doc)} for qid, doc in zip(qids, nlp.pipe(sents))]

In [9]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption']) for r in fixed)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-03-07 12:18:13.043967: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-07 12:18:13.044276: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-07 12:18:13.044289: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-07 12:18:13.076716: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

In [10]:
len([g for g in res if len(g['matches']) > 0])

1788

In [16]:
roots = [{'_id': qid, 'roots': [c.text for c in lt.find_root_nsubj(doc)]} for qid, doc in zip(qids, nlp.pipe(sents))]

In [17]:
roots[0]

{'_id': 326985, 'roots': ['is', 'rate']}

In [19]:
RULE = 'NNS_IN_NNP_VBZ'
to_ignore = [q['_id'] for q in roots if q['roots'][0] == 'is']

In [22]:
to_ignore = set(to_ignore)

In [23]:
to_ignore = [c for c in schools if c['_id'] in to_ignore]

In [27]:
for t in to_ignore:
    ig = t.get('ignore', [RULE])
    t['ignore'] = ig

In [28]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in to_ignore]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 1762,
 'nModified': 1762,
 'nRemoved': 0,
 'upserted': []}

In [30]:
typos = lt.filter_lt_messages(G, message='Possible spelling mistake found')
lt.get_repl_values(typos)

{'Bank', 'Bank are', 'Bank bare', 'Banks', 'Banks are'}

In [31]:
typos[0]

{'_id': 4680,
 'question_string': 'What is the net disbursements of loan from World Bankb in 1977 ?',
 'template': 'data_retrieval',
 'answer': '3164000',
 'type': 'dot',
 'template_id': 2,
 'caption': 'The net disbursements of loan from World Bankb in 1977 are 3164000.',
 'matches': [{'message': 'Possible spelling mistake found.',
   'shortMessage': 'Spelling mistake',
   'replacements': [{'value': 'Bank'}, {'value': 'Banks'}],
   'offset': 41,
   'length': 5,
   'context': {'text': '...he net disbursements of loan from World Bankb in 1977 are 3164000.',
    'offset': 43,
    'length': 5},
   'sentence': 'The net disbursements of loan from World Bankb in 1977 are 3164000.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'MORFOLOGIK_RULE_EN_US',
    'description': 'Possible spelling mistake',
    'issueType': 'misspelling',
    'category': {'id': 'TYPOS', 'name': 'Possible Typo'}},
   'ignoreForIncompleteSentence': False,
   'contextForSureMatch': 0}]}

In [38]:
for t in typos:
    t['caption'] = re.sub(r'\bBankb\b', 'Bank', t['caption'])

In [40]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption']) for r in typos)
len([g for g in res if len(g['matches']) > 0])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 1693 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1748 out of 1748 | elapsed:    3.4s finished


0

In [41]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in typos]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 1748,
 'nModified': 1748,
 'nRemoved': 0,
 'upserted': []}

In [42]:
services = lt.filter_lt_messages(G, message='services')
services[0]

{'_id': 120614,
 'question_string': 'Does the percentage of population using sanitation services in Togo monotonically increase over the years ?',
 'template': 'compound',
 'answer': 'No',
 'type': 'dot',
 'template_id': 1,
 'caption': 'The percentage of population the Using sanitation services in Togo does not increase over the years.',
 'matches': [{'message': 'With the plural noun ‘services’, the verb inflection ‘does’ is not correct.',
   'shortMessage': 'Agreement error',
   'replacements': [{'value': 'do'}, {'value': 'did'}],
   'offset': 67,
   'length': 4,
   'context': {'text': '...n the Using sanitation services in Togo does not increase over the years.',
    'offset': 43,
    'length': 4},
   'sentence': 'The percentage of population the Using sanitation services in Togo does not increase over the years.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'NNS_IN_NNP_VBZ',
    'subId': '1',
    'sourceFile': 'grammar.xml',
    'description': 'Use the base form or the past 

In [47]:
for s in services:
    s['caption'] = re.sub(r'\bthe Using\b', 'using', s['caption'])

In [48]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption']) for r in services)
len([g for g in res if len(g['matches']) > 0])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 2008 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 2137 out of 2137 | elapsed:    4.9s finished


2122

In [55]:
res[:5]

[{'question_id': 120614, 'matches': []},
 {'question_id': 122405, 'matches': []},
 {'question_id': 125750, 'matches': []},
 {'question_id': 126872, 'matches': []},
 {'question_id': 127965,
  'matches': [{'message': 'With the plural noun ‘services’, the verb inflection ‘does’ is not correct.',
    'shortMessage': 'Agreement error',
    'replacements': [{'value': 'do'}, {'value': 'did'}],
    'offset': 56,
    'length': 4,
    'context': {'text': '...ayments for transport services in India does not monotonically grow over the years.',
     'offset': 43,
     'length': 4},
    'sentence': 'The balance of payments for transport services in India does not monotonically grow over the years.',
    'type': {'typeName': 'Other'},
    'rule': {'id': 'NNS_IN_NNP_VBZ',
     'subId': '1',
     'sourceFile': 'grammar.xml',
     'description': 'Use the base form or the past tense with a plural noun',
     'issueType': 'grammar',
     'category': {'id': 'GRAMMAR', 'name': 'Grammar'}},
    'ignoreForIn

In [64]:
qids = [g['question_id'] for g in res]
sents = [g['caption'] for g in services]

roots = []
for qid, doc in zip(qids, nlp.pipe(sents)):
    try:
        roots.append({'_id': qid, 'roots': lt.find_root_nsubj(doc)})
    except IndexError:
        print(doc)

The mean per year amount collected by government from taxes on goods and services in Azerbaijan is 2597133333.3333335.
The mean per year amount collected by government from taxes on goods and services in Uruguay is 79578336323.5457.
The mean per year amount collected by government from taxes on goods and services in Denmark is 258821666666.66666.
The mean per year amount collected by government from taxes on goods and services in Slovenia is 3604938738.942835.
The mean per year amount collected by government from taxes on goods and services in Peru is 15923363333.333334.
The mean per year amount collected by government from taxes on goods and services in Georgia is 2811450000.0.


In [73]:
for r in roots:
    try:
        r['roots'] = tuple(c for c in r['roots'])
    except AttributeError:
        continue

In [74]:
set([r['roots'] for r in roots])

{('exceeds', '%'),
 ('exceeds', 'Balance'),
 ('exceeds', 'Revenue'),
 ('exceeds', 'balance'),
 ('exceeds', 'percentage'),
 ('grow', 'amount'),
 ('grow', 'balance'),
 ('grow', 'percentage'),
 ('grows', 'amount'),
 ('grows', 'balance'),
 ('grows', 'percentage'),
 ('increase', 'amount'),
 ('increase', 'balance'),
 ('increase', 'percentage'),
 ('is', '%'),
 ('is', 'Balance'),
 ('is', 'OECD'),
 ('is', 'Revenue'),
 ('is', 'computer'),
 ('is', 'difference'),
 ('is', 'sum'),
 (-16.1904848642, None)}

In [12]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'caption': r['caption']}}) for r in fixed]

In [13]:
wres = db.captions.bulk_write(updates)
wres.bulk_api_results

AttributeError: 'BulkWriteResult' object has no attribute 'bulk_api_results'

In [14]:
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 37970,
 'nModified': 14761,
 'nRemoved': 0,
 'upserted': []}

In [15]:
len(fixed)

37970

In [16]:
14761 / _

0.38875427969449566

In [17]:
G = [d for d in db.captions.find({"matches.0": {"$exists": True}})]
len(G)

25526

In [18]:
ctr = lt.err_counter(G, top=False)
ctr.most_common(5)

[('With the plural noun ‘tourists’, the verb inflection ‘is’ is not correct.',
  2129),
 ('With the plural noun ‘schools’, the verb inflection ‘is’ is not correct.',
  1762),
 ('Possible spelling mistake found.', 1748),
 ('With the plural noun ‘services’, the verb inflection ‘is’ is not correct.',
  1646),
 ('With the plural noun ‘firms’, the verb inflection ‘is’ is not correct.',
  1445)]

In [19]:
msg = 'With the plural noun ‘tourists’, the verb inflection ‘is’ is not correct.'
tourists = lt.filter_lt_messages(G, message=msg)
tourists[0]

{'_id': 121134,
 'question_string': 'What is the total amount spent on the travel items of international tourists in Spain in the graph ?',
 'template': 'arithmetic',
 'answer': '106210000000',
 'type': 'dot',
 'template_id': 2,
 'caption': 'In the graph, the total amount spent on the travel items of international tourists in Spain is 106210000000.',
 'matches': [{'message': 'With the plural noun ‘tourists’, the verb inflection ‘is’ is not correct.',
   'shortMessage': 'Agreement error',
   'replacements': [{'value': 'are'},
    {'value': 'was'},
    {'value': 'were',
     'shortDescription': "second-person singular simple past of 'be', e.g. 'you were'"}],
   'offset': 92,
   'length': 2,
   'context': {'text': '...tems of international tourists in Spain is 106210000000.',
    'offset': 43,
    'length': 2},
   'sentence': 'In the graph, the total amount spent on the travel items of international tourists in Spain is 106210000000.',
   'type': {'typeName': 'Other'},
   'rule': {'id': '

In [21]:
s = nlp('In the graph, the total amount spent on the travel items of international tourists in Spain is 106210000000.')
verb, nsubj = lt.find_root_nsubj(s)

In [22]:
verb, nsubj

(is, amount)

In [23]:
verb_nsubj = [lt.find_root_nsubj(doc) for doc in nlp.pipe([t['caption'] for t in tourists])]

In [26]:
verbs = set([k[0].text for k in verb_nsubj])
subjs = set([k[1].text for k in verb_nsubj])

In [27]:
verbs

{'is'}

In [28]:
subjs

{'amount', 'difference', 'sum'}

In [32]:
RULE = 'NNS_IN_NNP_VBZ'
for t in tourists:
    ig = t.get('ignore', [RULE])
    t['ignore'] = ig

In [34]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r['ignore']}}) for r in tourists]
wres = db.captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 2129,
 'nModified': 2129,
 'nRemoved': 0,
 'upserted': []}

In [36]:
G = [d for d in db.captions.find({"matches.0": {"$exists": True}})]
len(G)

23397

In [37]:
ctr = lt.err_counter(G, top=False)
ctr.most_common(5)

[('With the plural noun ‘schools’, the verb inflection ‘is’ is not correct.',
  1762),
 ('Possible spelling mistake found.', 1748),
 ('With the plural noun ‘services’, the verb inflection ‘is’ is not correct.',
  1646),
 ('With the plural noun ‘firms’, the verb inflection ‘is’ is not correct.',
  1445),
 ('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302)]

In [39]:
ctr.most_common()

[('With the plural noun ‘schools’, the verb inflection ‘is’ is not correct.',
  1762),
 ('Possible spelling mistake found.', 1748),
 ('With the plural noun ‘services’, the verb inflection ‘is’ is not correct.',
  1646),
 ('With the plural noun ‘firms’, the verb inflection ‘is’ is not correct.',
  1445),
 ('The singular proper name ‘Republic’ must be used with a third-person or a past tense verb.',
  1302),
 ('The word ‘vitamin’ is normally not capitalized in the expression “vitamin A”',
  1282),
 ('Use a comma before “and” if it connects two independent clauses (unless they are closely connected and short).',
  1132),
 ('With the plural noun ‘materials’, the verb inflection ‘is’ is not correct.',
  1005),
 ('With the plural noun ‘products’, the verb inflection ‘is’ is not correct.',
  921),
 ('With the plural noun ‘exports’, the verb inflection ‘is’ is not correct.',
  876),
 ('Possible typo: you repeated a word', 847),
 ('The usual collocation for “Interest” is “in”, not “by”.', 795),

In [41]:
caps = [g['caption'] for g in G]
fixed = [{'_id': r['_id'], 'caption': fix(doc)} for r, doc in zip(G, nlp.pipe(caps))]

In [42]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption']) for r in fixed)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-02-06 12:34:15.720433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-06 12:34:15.720873: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-06 12:34:15.720885: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-02-06 12:34:15.774715: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

In [43]:
len([g for g in res if len(g['matches']) > 0])

23397

In [44]:
len(res)

23397